In [47]:
import tsplib95
import numpy as np
problem = tsplib95.load('../data/bays29.tsp')

In [48]:
EVAP_RATE = 0.5 #0.2 or 0.8 seems to be best (see graph)
NBR_ANTS = 29
#alpha is relative importance of pheromone
ALPHA = 1
#beta is relative importance heuristic
BETA = 5
Q = 1.5
#Q is the constant used to update the pheromones (Q/L)
ITERATIONS = 500
np.random.seed(0)

DISTANCES = np.array([[problem.get_weight(i, j) for j in range(1, problem.dimension + 1)] for i in range(1, problem.dimension + 1)])
DISTANCES[DISTANCES == 0] = 1
DISTANCES_POWERED = 1/ np.power(DISTANCES, BETA) 
pheromones =np.full((DISTANCES.shape), 0.1)

In [49]:
weight = problem.trace_canonical_tour()
print(weight)

5752


In [50]:
visited_cities = np.zeros((NBR_ANTS, 1), int)
visited_cities.shape


(29, 1)

In [51]:
cities_to_visit = np.ones((problem.dimension,NBR_ANTS), dtype=int)
cities_to_visit[0]= 0
cities_to_visit.shape

(29, 29)

In [52]:
def get_next_cities(current_cities, cities_to_visit, pheromones):
    teta_nues = np.array([  pheromones[city] **ALPHA * cities_to_visit.T[i] * DISTANCES_POWERED[city]  for i,city in enumerate(current_cities) ])
    probabilities = teta_nues / teta_nues.sum(axis=1)[:,None]
    next_cities = np.array([np.random.choice(teta_nues.shape[1], p=probabilities[i]) for i in range(teta_nues.shape[0])])
    return next_cities

In [53]:
# each iteration upadate pheromones with
# new_pheromone = Q/weight(edge)
# pheromone = (1 - EVAP_RATE) * old_pheromone + new_pheromone

def update_pheromones(pheromones, current_cities, next_cities):
    #evaporate pheromones
    pheromones *= EVAP_RATE
    #add pheromones to travelled edges
    delta_pheromones = Q / DISTANCES[current_cities, next_cities]

    pheromones[current_cities, next_cities] += delta_pheromones
    pheromones[next_cities, current_cities] += delta_pheromones

    return pheromones

In [54]:
for _ in range(ITERATIONS) :
   # Initialisation, we start at city 0
    visited_cities = np.zeros((NBR_ANTS, 1), int)
    cities_to_visit = np.ones((problem.dimension,NBR_ANTS), dtype=int)
    cities_to_visit[0]= 0
    current_cities = np.zeros(NBR_ANTS,dtype=int)

    for _ in range(problem.dimension - 1):
        next_cities = get_next_cities(current_cities, cities_to_visit, pheromones)
        update_pheromones(pheromones, current_cities, next_cities)
        current_cities = next_cities
        cities_to_visit[current_cities, np.arange(len(current_cities))] = 0
        visited_cities = np.append(visited_cities, current_cities.reshape(NBR_ANTS,1), axis=1)


In [55]:
def calculate_distance(visited_cities):
    distances = np.array([DISTANCES[visited_cities[i], visited_cities[i+1]] for i in range(len(visited_cities)-1)])
    return distances.sum(axis=0)



In [56]:
solution = tsplib95.load('../data/bays29.opt.tour')
print("My tour's length: ", calculate_distance(visited_cities[0]))

print("optimal tour's length: ",problem.trace_tours(solution.tours))

visited_cities +=1

print(visited_cities[1])

print(solution.tours[0])

My tour's length:  2194
optimal tour's length:  [2020]
[ 1 28  6 12  9  5 26 29  3 17 18 14 22 11 15  4 10 20  2 21 13 16 19 25
  7 23  8 24 27]
[1, 28, 6, 12, 9, 5, 26, 29, 3, 2, 20, 10, 4, 15, 18, 17, 14, 22, 11, 19, 25, 7, 23, 27, 8, 24, 16, 13, 21]
